In [ ]:
import os
import json
import pandas as pd
import psycopg as psycopg
from dotenv import load_dotenv

## Helper functions

In [ ]:
def queries(query):
    load_dotenv()
    conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
    cur = conn.cursor()

    cur.execute(query)
    temp = cur.fetchall()

    cur.close()
    conn.close()

    return temp

## DROP and CREATE DATABASE yelp

Make sure to put in your password for your postgres user in code/password.txt and upload the json file into the data folder then just run the cells below.

__CAUTION: ONLY RUN THE CELL BELOW IF YOU DON'T HAVE YELP DATABASE UP. RUNNING THE CELL BELOW WILL REMOVE THE DATABASE WHICH WILL REMOVE ALL RELATIONS THAT EXIST WITHIN YELP.__

In [ ]:
load_dotenv()
conn = psycopg.connect(f"dbname='postgres' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

cur.execute("""DROP DATABASE IF EXISTS yelp;""")
cur.execute("""CREATE DATABASE yelp;""")

cur.close()
conn.close()

## Create and insert into business table

In [4]:
try:
    business_data
except NameError:
    business_data = []
    with open('../data/yelp_academic_dataset_business.json', 'r') as file:
        for line in file:
            business_data.append(json.loads(line))

In [ ]:
conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS businesses;
""")

cur.execute("""
CREATE TABLE businesses (
business_id VARCHAR(22) PRIMARY KEY,
name TEXT NOT NULL,
address VARCHAR(255),
city VARCHAR(100),
state VARCHAR(20),
postal_code VARCHAR(15),
latitude FLOAT,
longitude FLOAT,
stars FLOAT CHECK (stars >= 0 AND stars <= 5),
review_count INT CHECK(review_count >= 0),
is_open BOOLEAN,
attributes JSONB,
categories TEXT[],
hours JSONB
);""")

cur.close()
conn.close()

In [ ]:
conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

for i in range(len(business_data)):
    cur.execute("""
    INSERT INTO businesses (business_id,
                          name,
                          address,
                          city,
                          state,
                          postal_code,
                          latitude,
                          longitude,
                          stars,
                          review_count,
                          is_open,
                          attributes,
                          categories,
                          hours)
    VALUES (%(business_id)s,
            %(name)s,
            %(address)s,
            %(city)s,
            %(state)s,
            %(postal_code)s,
            %(latitude)s,
            %(longitude)s,
            %(stars)s,
            %(review_count)s,
            %(is_open)s,
            %(attributes)s,
            %(categories)s,
            %(hours)s);
    """,
    {'business_id': business_data[i]['business_id'],
     'name': business_data[i]['name'],
     'address': business_data[i]['address'],
     'city': business_data[i]['city'],
     'state': business_data[i]['state'],
     'postal_code': business_data[i]['postal_code'],
     'latitude': business_data[i]['latitude'],
     'longitude': business_data[i]['longitude'],
     'stars': business_data[i]['stars'],
     'review_count': business_data[i]['review_count'],
     'is_open': bool(business_data[i]['is_open']),
     'attributes': json.dumps(business_data[i]['attributes']),
     'categories': [business_data[i]['categories']],
     'hours': json.dumps(business_data[i]['hours'])
    })

cur.close()
conn.close()

## Create and insert into user table

In [7]:
try:
    user_data
except NameError:
    user_data = []
    with open('../data/yelp_academic_dataset_user.json', 'r') as file:
        for line in file:
            user_data.append(json.loads(line))

In [ ]:
conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS users;
""")

cur.execute("""
CREATE TABLE users (
user_id VARCHAR(22) PRIMARY KEY,
name TEXT NOT NULL,
review_count INT CHECK (review_count >= 0),
yelping_since DATE,
friends TEXT[],
useful INT CHECK (useful >= 0),
funny INT CHECK (funny >= 0),
cool INT CHECK (cool >= 0),
fans INT CHECK (fans >= 0),
elite TEXT[],
average_stars FLOAT CHECK (average_stars >= 0 AND average_stars <= 5),
compliment_hot INT CHECK (compliment_hot >= 0),
compliment_more INT CHECK (compliment_more >= 0),
compliment_profile INT CHECK (compliment_profile >= 0),
compliment_cute INT CHECK (compliment_cute >= 0),
compliment_list INT CHECK (compliment_list >= 0),
compliment_note INT CHECK (compliment_note >= 0),
compliment_plain INT CHECK (compliment_plain >= 0),
compliment_cool INT CHECK (compliment_cool >= 0),
compliment_funny INT CHECK (compliment_funny >= 0),
compliment_writer INT CHECK (compliment_writer >= 0),
compliment_photos INT CHECK (compliment_photos >= 0)
);""")

cur.close()
conn.close()

In [ ]:
conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

for i in range(len(user_data)):
    cur.execute("""
    INSERT INTO users (user_id,
                        name,
                        review_count,
                        yelping_since,
                        friends,
                        useful,
                        funny,
                        cool,
                        fans,
                        elite,
                        average_stars,
                        compliment_hot,
                        compliment_more,
                        compliment_profile,
                        compliment_cute,
                        compliment_list,
                        compliment_note,
                        compliment_plain,
                        compliment_cool,
                        compliment_funny,
                        compliment_writer,
                        compliment_photos)
    VALUES (%(user_id)s,
            %(name)s,
            %(review_count)s,
            %(yelping_since)s,
            %(friends)s,
            %(useful)s,
            %(funny)s,
            %(cool)s,
            %(fans)s,
            %(elite)s,
            %(average_stars)s,
            %(compliment_hot)s,
            %(compliment_more)s,
            %(compliment_profile)s,
            %(compliment_cute)s,
            %(compliment_list)s,
            %(compliment_note)s,
            %(compliment_plain)s,
            %(compliment_cool)s,
            %(compliment_funny)s,
            %(compliment_writer)s,
            %(compliment_photos)s);
    """,
    {'user_id': user_data[i]['user_id'],
     'name': user_data[i]['name'],
     'review_count': user_data[i]['review_count'],
     'yelping_since': user_data[i]['yelping_since'],
     'friends': [user_data[i]['friends']],
     'useful': user_data[i]['useful'],
     'funny': user_data[i]['funny'],
     'cool': user_data[i]['cool'],
     'fans': user_data[i]['fans'],
     'elite': [user_data[i]['elite']],
     'average_stars': user_data[i]['average_stars'],
     'compliment_hot': user_data[i]['compliment_hot'],
     'compliment_more': user_data[i]['compliment_more'],
     'compliment_profile': user_data[i]['compliment_profile'],
     'compliment_cute': user_data[i]['compliment_cute'],
     'compliment_list': user_data[i]['compliment_list'],
     'compliment_note': user_data[i]['compliment_note'],
     'compliment_plain': user_data[i]['compliment_plain'],
     'compliment_cool': user_data[i]['compliment_cool'],
     'compliment_funny': user_data[i]['compliment_funny'],
     'compliment_writer': user_data[i]['compliment_writer'],
     'compliment_photos': user_data[i]['compliment_photos']
    })

cur.close()
conn.close()

## Create and insert into review table

In [10]:
try:
    review_data
except NameError:
    review_data = []
    with open('../data/yelp_academic_dataset_review.json', 'r') as file:
        for line in file:
            review_data.append(json.loads(line))

In [ ]:
conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS temp_reviews;
""")

cur.execute("""
CREATE TABLE temp_reviews (
review_id VARCHAR(22) PRIMARY KEY,
user_id VARCHAR(22),
business_id VARCHAR(22),
stars INT CHECK (stars >= 0 AND stars <= 5),
date DATE,
text TEXT,
useful INT,
funny INT,
cool INT
);""")

cur.close()
conn.close()

In [ ]:
conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS reviews;
""")

cur.execute("""
CREATE TABLE reviews (
review_id VARCHAR(22) PRIMARY KEY,
user_id VARCHAR(22) REFERENCES users (user_id) ON DELETE CASCADE ON UPDATE CASCADE,
business_id VARCHAR(22) REFERENCES businesses (business_id) ON DELETE CASCADE ON UPDATE CASCADE,
stars INT CHECK (stars >= 0 AND stars <= 5),
date DATE,
text TEXT,
useful INT,
funny INT,
cool INT
);""")

cur.close()
conn.close()

In [ ]:
conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

for i in range(len(review_data)):
    cur.execute("""
    INSERT INTO temp_reviews (review_id,
                        user_id,
                        business_id,
                        stars,
                        date,
                        text,
                        useful,
                        funny,
                        cool)
    VALUES (%(review_id)s,
            %(user_id)s,
            %(business_id)s,
            %(stars)s,
            %(date)s,
            %(text)s,
            %(useful)s,
            %(funny)s,
            %(cool)s);
    """,
    {'review_id': review_data[i]['review_id'],
     'user_id': review_data[i]['user_id'],
     'business_id': review_data[i]['business_id'],
     'stars': review_data[i]['stars'],
     'date': review_data[i]['date'],
     'text': review_data[i]['text'],
     'useful': review_data[i]['useful'],
     'funny': review_data[i]['funny'],
     'cool': review_data[i]['cool']
    })

cur.close()
conn.close()

In [ ]:
valid_id = queries("""
SELECT tr.review_id, tr.user_id, tr.business_id, tr.stars, tr.date, tr.text, tr.useful, tr.funny, tr.cool
FROM temp_reviews tr
WHERE tr.user_id IN (SELECT user_id FROM users)
AND tr.business_id IN (SELECT business_id FROM businesses);
""")

conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

cur.executemany("""
INSERT INTO reviews (review_id,
                        user_id,
                        business_id,
                        stars,
                        date,
                        text,
                        useful,
                        funny,
                        cool)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
""",
valid_id
)

cur.close()
conn.close()

In [15]:
queries("""
SELECT table_name, table_type
FROM information_schema.tables
WHERE table_schema = 'public';
""")

[('businesses', 'BASE TABLE'),
 ('users', 'BASE TABLE'),
 ('temp_reviews', 'BASE TABLE'),
 ('reviews', 'BASE TABLE')]

In [ ]:
conn = psycopg.connect(f"dbname='yelp' user=postgres password={os.getenv('PSQL_PASSWORD')}", autocommit=True)
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS temp_reviews;
""")

cur.close()
conn.close()

## Commands to check if any other users is using the database incase you need to drop the database

In [6]:
queries("""
SELECT *, pg_terminate_backend(pid)
FROM pg_stat_activity 
WHERE pid <> pg_backend_pid()
AND datname = 'yelp';
""")

[]

In [5]:
queries("""
SELECT pg_terminate_backend(pg_stat_activity.pid)
FROM pg_stat_activity
WHERE pg_stat_activity.datname = 'yelp'
  AND pid <> pg_backend_pid();
""")

[]